In [3]:
import dagshub
import mlflow
#dagshub.init(url="https://dagshub.com/PacoTinoco/Proyecto_Final_CDD", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="google_stock")

file:///C:/Users/almen/Documents/dev/samsara/pythonProject/app/tmp/mlruns


<Experiment: artifact_location='file:///C:/Users/almen/Documents/dev/samsara/pythonProject/app/tmp/mlruns/888048669364449520', creation_time=1730587820702, experiment_id='888048669364449520', last_update_time=1730587820702, lifecycle_stage='active', name='google_stock', tags={}>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

google_stock = yf.download('GOOGL', start='2015-01-01', end='2024-01-01')

[*********************100%***********************]  1 of 1 completed


In [4]:
# Definir X e y
X = google_stock.drop(columns=["Close"])
y = google_stock["Close"]
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño del conjunto de prueba:", X_val.shape, y_val.shape)


Tamaño del conjunto de entrenamiento: (1811, 5) (1811, 1)
Tamaño del conjunto de prueba: (453, 5) (453, 1)


In [5]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib
from sklearn.metrics import  root_mean_squared_error
import numpy as np

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [9]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [11]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "Google Stock validation",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    import joblib
    joblib.dump(booster, 'models/xgboost_model_goog.pkl')
    mlflow.log_artifact('models/xgboost_model_goog.pkl', artifact_path="model")

2024/11/02 16:59:37 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.1, but the installed version is 2.1.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


[0]	validation-rmse:18.57522                          
[1]	validation-rmse:9.43738                           
[2]	validation-rmse:4.80348                           
[3]	validation-rmse:2.45919                           
[4]	validation-rmse:1.28834                           
[5]	validation-rmse:0.71192                           
[6]	validation-rmse:0.45053                           
[7]	validation-rmse:0.34580                           
[8]	validation-rmse:0.31072                           
[9]	validation-rmse:0.30028                           
[10]	validation-rmse:0.29743                          
[11]	validation-rmse:0.29652                          
[12]	validation-rmse:0.29670                          
[13]	validation-rmse:0.29666                          
[14]	validation-rmse:0.29647                          
[15]	validation-rmse:0.29661                          
[16]	validation-rmse:0.29667                          
[17]	validation-rmse:0.29675                          
[18]	valid

2024/11/02 16:59:38 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 16:59:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 16:59:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

[0]	validation-rmse:23.62358                                                   
[1]	validation-rmse:15.25648                                                   
[2]	validation-rmse:9.85475                                                    
[3]	validation-rmse:6.36725                                                    
[4]	validation-rmse:4.11871                                                    
[5]	validation-rmse:2.66815                                                    
[6]	validation-rmse:1.73730                                                    
[7]	validation-rmse:1.14005                                                    
[8]	validation-rmse:0.76882                                                    
[9]	validation-rmse:0.53999                                                    
[10]	validation-rmse:0.40813                                                   
[11]	validation-rmse:0.33737                                                   
[12]	validation-rmse:0.30039            

2024/11/02 16:59:49 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 16:59:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:59:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 16:59:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

...

2024/11/02 17:00:56 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 17:00:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:00:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 17:01:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

[0]	validation-rmse:24.86955                                                    
[1]	validation-rmse:16.91074                                                    
[2]	validation-rmse:11.50287                                                    
[3]	validation-rmse:7.82110                                                     
[4]	validation-rmse:5.32262                                                     
[5]	validation-rmse:3.62607                                                     
[6]	validation-rmse:2.47436                                                     
[7]	validation-rmse:1.69393                                                     
[8]	validation-rmse:1.17239                                                     
[9]	validation-rmse:0.82647                                                     
[10]	validation-rmse:0.60091                                                    
[11]	validation-rmse:0.46157                                                    
[12]	validation-rmse:0.37833

2024/11/02 17:01:04 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/11/02 17:01:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:01:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/11/02 17:01:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: 

100%|██████████| 10/10 [01:34<00:00,  9.50s/trial, best loss: 0.2559164272635569]
[0]	validation-rmse:33.63181
[1]	validation-rmse:30.91505
[2]	validation-rmse:28.41999
[3]	validation-rmse:26.13177
[4]	validation-rmse:24.02355
[5]	validation-rmse:22.08911
[6]	validation-rmse:20.29920
[7]	validation-rmse:18.66303
[8]	validation-rmse:17.16210
[9]	validation-rmse:15.78163
[10]	validation-rmse:14.51127
[11]	validation-rmse:13.33714
[12]	validation-rmse:12.26276
[13]	validation-rmse:11.27071
[14]	validation-rmse:10.36589
[15]	validation-rmse:9.53131
[16]	validation-rmse:8.76536
[17]	validation-rmse:8.05838
[18]	validation-rmse:7.41160
[19]	validation-rmse:6.81444
[20]	validation-rmse:6.26708
[21]	validation-rmse:5.76315
[22]	validation-rmse:5.29942
[23]	validation-rmse:4.87508
[24]	validation-rmse:4.48486
[25]	validation-rmse:4.12510
[26]	validation-rmse:3.79447
[27]	validation-rmse:3.49042
[28]	validation-rmse:3.21152
[29]	validation-rmse:2.95453
[30]	validation-rmse:2.71870
[31]	validatio

2024/11/02 17:01:13 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/11/02 17:01:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\almen\Documents\dev\samsara\pythonProject\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:01:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/11/02 17:01:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [12]:
best_params

{'learning_rate': 0.0810929814758782,
 'max_depth': 40,
 'min_child_weight': 3.4635416745403758,
 'reg_alpha': 0.06794451678204419,
 'reg_lambda': 0.3098339345911543,
 'seed': 42,
 'objective': 'reg:squarederror'}

In [25]:
import mlflow

In [26]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="google-stock-model"
)

Ingrese el run_id 5fc0756e7f73427cb4b6a8bed889a20b


Successfully registered model 'google-stock-model'.
2024/09/28 13:10:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: google-stock-model, version 1
Created version '1' of model 'google-stock-model'.


In [27]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="google-stock-model",
    description="Model registry for the google stock prediction",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="google-stock-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="google-stock-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1727550617515, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-28 13:12:02.669914', last_updated_timestamp=1727550720469, name='google-stock-model', run_id='5fc0756e7f73427cb4b6a8bed889a20b', run_link='', source='mlflow-artifacts:/6ec4ca913e1442d7a8287af46c3fa3b1/5fc0756e7f73427cb4b6a8bed889a20b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [28]:
import mlflow.pyfunc

model_name = "google-stock-model"
alias = "champion"

model_uri = f"db_models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([143.453   ,  72.09996 ,  36.609688,  39.58499 ,  28.274122,
       101.74806 ,  55.14543 , 135.74756 ,  27.341633, 100.93057 ,
        98.4985  ,  33.11652 ,  41.786972,  53.575924, 104.2153  ,
       144.40746 ,  78.695564, 109.47353 ,  37.52547 , 109.6177  ,
       114.617744,  74.52254 , 137.93652 ,  88.22144 ,  55.855217,
       141.85316 ,  59.42868 ,  35.214108,  60.18714 ,  66.35824 ,
        27.66563 ,  99.641205, 112.27483 ,  39.037693, 120.42924 ,
       135.73648 ,  59.399837,  68.086845,  42.42141 ,  52.24469 ,
       129.81413 , 140.84976 ,  39.980915,  54.611214, 118.23367 ,
        75.56147 , 147.15639 ,  46.274845, 102.66855 ,  41.178814,
        27.851946, 145.38954 , 133.83778 ,  40.492138, 145.56761 ,
        90.6261  ,  59.570694,  48.011665,  73.25103 , 127.962166,
        52.441124,  30.575087,  90.829254,  71.52325 , 115.75765 ,
        38.918068, 104.460266,  36.359734,  52.231182, 129.63306 ,
       130.11194 ,  56.01812 , 127.7084  ,  52.599   ,  35.273